In [1]:
import sys

from dask.diagnostics import ProgressBar
import xarray as xr
import numpy as np
import datetime 
import time
from datetime import datetime 
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd


import pyOptimalEstimation as pyOE

import cartopy
import cartopy.crs as ccrs
from cmocean import cm as cmo

import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from glob import glob


import gc, psutil

In [2]:
#sys.path.append('/nobackup/users/echeverr/Git/SatOpEst/support') # where supporting_routines_m live
sys.path.append('/home/mario/Documents/work/code/git/SatOpEst/support') # where supporting_routines_m live

import supporting_routines_m 

import os

#rttov_installdir = '/usr/people/echeverr/Documents/code/nwpsaf/rttov13'
rttov_installdir = '/home/mario/myLibs/rrtov13/rttov130'

sys.path.append(rttov_installdir+'/wrapper')
import pyrttov



In [3]:
p = psutil.Process()

#BT_dir = '/nobackup/users/echeverr/data/cmsaf/ssmis/F16/'
BT_dir = '/home/mario/Data/CMSAF/ssims/F16/'

BT_dir2 = BT_dir+'/V1/'

In [4]:
#BT_file = '*.nc'
#BT_file = 'BTRin20140909000000324SSF1601GL' #BTRin20140910000000324SSF1601GL
#BT_file = 'BTRin20140910000000324SSF1601GL'
BT_file = 'BTRin20140916000000324SSF1601GL'

BT_file2 = BT_file+'_v1'


# Chunking the dataset (for 30 mins 10,10 works well (in my laptop, Mario); 
# if increase minutes times x, then increase chunk_size_time times x as well (avoids memory problems))
chunk_size_time =  10 # 420 for half day
chunk_size_s_a_t = 10

# user input:
#init_date = np.datetime64('2014-09-09T00:25:00.000') 
#end_date = np.datetime64('2014-09-09T00:30:00.000')
init_date = np.datetime64('2014-09-16T00:00:00.000') 
end_date = np.datetime64('2014-09-16T23:59:59.000')

In [5]:
import pyproj

import pyresample
from pyresample import create_area_def, load_area, data_reduce, utils, AreaDefinition
from pyresample.geometry import SwathDefinition
from pyresample.kd_tree import resample_nearest

In [6]:
# Reading Netcdf using xarray:
def read_netcdfs(files, dim, transform_func=None, groups = None):
    def process_one_path(path):
        # use a context manager, to ensure the file gets closed after use
        with xr.open_dataset(path, group = groups) as ds:
            # transform_func should do some sort of selection or
            # aggregation
            if transform_func is not None:
                ds = transform_func(ds)
            # load all data from the transformed dataset, to ensure we can
            # use it after closing each original file
            ds.load()
            return ds

    paths = sorted(glob(files))
    datasets = [process_one_path(p) for p in paths]
    combined = xr.concat(datasets, dim)
    return combined

# Apply usual flags to CMSAF dataset (per scene)
# Author: M. Echeverri, March 2021.
# TODO:
# - Add list of flags that the user wants to apply
# - Add input checks


def apply_scene_flags1(scene_BT, BT_attributes):
    
    for i, scene in enumerate(scene_BT):
        
        scene_BT[i] = scene_BT[i].where(scene.qc_fov==0) # Apply 'qc_fov' flag 
        
        # TODO: ical offsets are applied only to ssmis (they are all referenced to ssmi f11, I think, check)
        attrs = scene_BT[i]['tb'].attrs
        scene_BT[i]['tb'] = scene_BT[i].tb + scene_BT[i].ical # Apply intercalibration offsets         
        attrs['long_name'] = 'brightness temperature after ical'
        scene_BT[i]['tb'].attrs = attrs # keep attributes after ical   
        
        j = 0
        for ch in scene.scene_channel.values:              # Apply 'qc_channel' flag 
            pos = (BT_attributes.qc_channel[:,scene.scene_channel[j]].values!=0)
            scene_BT[i]['tb'].values[pos,j,:] = np.nan
            j+=1
        
    return scene_BT



In [7]:
# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
BT_attributes = read_netcdfs(BT_dir+BT_file+'.nc', dim='time')
BT_attributes

<xarray.Dataset>
Dimensions:       (across_track: 360, channel: 26, time: 45505, date: 1)
Coordinates:
  * across_track  (across_track) float32 1.0 1.5 2.0 2.5 ... 179.5 180.0 180.5
  * channel       (channel) int32 1 2 3 4 5 6 7 8 9 ... 19 20 21 22 23 24 25 26
  * date          (date) datetime64[ns] 2014-09-16T12:00:00
  * time          (time) datetime64[ns] 2014-09-16 ... 2014-09-16T23:59:58
Data variables:
    central_freq  (time, channel) float64 50.3 52.8 53.6 ... 60.79 85.5 85.5
    channel_if1   (time, channel) float64 0.0 0.0 0.0 0.0 ... 0.3579 0.0 0.0
    channel_if2   (time, channel) float64 0.0 0.0 0.0 0.0 ... 0.016 0.05 0.0 0.0
    polarization  (time, channel) |S50 b'v' b'v' b'v' b'v' ... b'rc' b'v' b'h'
    channel_name  (time, channel) |S50 b'v50' b'v52' b'v53' ... b'v85' b'h85'
    tfrac         (time) float64 1.38e+05 3.6e+04 9.35e+05 ... 8.24e+05 7.12e+05
    rev           (time) float64 5.63e+04 5.63e+04 ... 5.631e+04 5.631e+04
    qc_scan       (time) float32 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    pflag         (time) float32 2.0 2.0 2.0 2.0 2.0 2.0 ... 0.0 0.0 0.0 0.0 0.0
    qc_channel    (time, channel) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rotation      (time, date) float64 31.6 31.6 31.6 31.6 ... 31.6 31.6 31.6
    md5           (time) |S50 b'af564efc7f1e92c4bd72d36dad2d1622' ... b'22bbf...
Attributes: (12/43)
    title:                       CM SAF FCDR of SSMIS brightness temperatures
    summary:                     This dataset contains Fundamental Climate Da...
    Conventions:                 CF-1.6,ACDD-1.3
    netcdf_library_version:      4.3.3.1 of Mar  7 2016 08:18:19 $
    institution:                 EUMETSAT/CMSAF
    project:                     Satellite Application Facility on Climate Mo...
    ...                          ...
    format_version:              1.1
    tdr_software_rev_num:        10.0
    date_created:                2017-01-11T16:04:14 UTC
    date_modified:               2017-01-11T16:04:14 UTC
    filename:                    BTRin20140916000000324SSF1601GL.nc
    history:                     2017-01-12T12:30:58 UTC: mci-sc.cmsaf_archiv...

In [8]:
scenes_list = ['scene_env1','scene_env2'] #,'scene_img1','scene_img2','scene_las','scene_uas']

scene_BT = []

# BT_attributes.qc_scan can be applied at the moment of the retrieval?

#for scene in scenes_list:        
#    scene_BT.append(xr.open_mfdataset(
#        BT_dir+BT_file, combine = 'nested', 
#        concat_dim='time', group = scene)) 
    
# use 1 day dataset only:    
for scene in scenes_list:        
    scene_BT.append(xr.open_dataset(
        BT_dir+BT_file+'.nc', group = scene))  

In [ ]:
# HERE SCENE_HOMOGENIZATION (i.e. resample to unique or reference swath)

In [9]:
# After all scenes are sampled on the same reference swath, we can concatenate the TB's
# so we end up with a single dataset

scene_BT_test = xr.concat(
    apply_scene_flags1(scene_BT, BT_attributes),
    dim='scene_channel').drop_vars(
    ['laz','qc_fov','ical','eia_norm'])

# Because of the way xarray.concat works "scene_channel" is introduced as dimension
# in variables that do not depend on it (lat, lon, eia and sft); this is removed by
# selecting only one "scene_channel" in each of those variables:
scene_BT_test['lat'] = scene_BT_test.lat[0,:,:] #.copy()
scene_BT_test['lon'] = scene_BT_test.lon[0,:,:] #.copy()
scene_BT_test['eia'] = scene_BT_test.eia[0,:,:] #.copy()
scene_BT_test['sft'] = scene_BT_test.sft[0,:,:] #.copy()

In [10]:
DS_CMSAF = scene_BT_test.assign_coords(
    time=(BT_attributes.time)).chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t}) 

In [11]:
scene_BT_test=None

In [12]:
DS_CMSAF

<xarray.Dataset>
Dimensions:             (time: 45505, scene_across_track: 90, scene_channel: 9)
Coordinates:
  * scene_across_track  (scene_across_track) int32 1 5 9 13 ... 345 349 353 357
  * scene_channel       (scene_channel) int64 11 12 13 14 15 16 17 24 25
  * time                (time) datetime64[ns] 2014-09-16 ... 2014-09-16T23:59:58
Data variables:
    lat                 (time, scene_across_track) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    lon                 (time, scene_across_track) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    eia                 (time, scene_across_track) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    sft                 (time, scene_across_track) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    tb                  (time, scene_channel, scene_across_track) float32 dask.array<chunksize=(10, 9, 10), meta=np.ndarray>
Attributes:
    title:                 Environmental Scene 1
    comment:               feedhorn channels: h19, v19, v22
    elevation_offset_deg:  0.4
    azimuth_offset_deg:    -0.3

In [15]:
# nearest to user input in dataset:
init_date = DS_CMSAF.time.sel(time=init_date, method = "nearest")
end_date = DS_CMSAF.time.sel(time=end_date, method = "nearest")

DS_CMSAF_ocean = (DS_CMSAF.sel(time=slice(init_date,end_date)
                             , scene_channel = [11,12,14,15] #slice(11,15)
                              ).where(DS_CMSAF.sft==0)).transpose(...,"scene_channel") 

# Keep track of some useful metadata:
DS_CMSAF_ocean['global_channel_ID'] = \
             BT_attributes.channel[DS_CMSAF_ocean.scene_channel].drop_vars('channel')
DS_CMSAF_ocean['central_freq'] = \
             BT_attributes['central_freq'][0, DS_CMSAF_ocean['scene_channel']]
DS_CMSAF_ocean['polarization'] = \
             BT_attributes['polarization'][0, DS_CMSAF_ocean['scene_channel']]

# Add channel's uncertainty (useful for optimal estimation retrievals):
DS_CMSAF_ocean['channel_uncertainty'] = xr.DataArray(
                data   = np.array([2.4, 1.27, 3.0, 1.34,], dtype = np.float32),   # \
                dims   = ['scene_channel'],
                coords = {'scene_channel': DS_CMSAF_ocean.scene_channel,},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Uncertainty of observations (i.e. Noise)',
                    'units'     : 'K'
                    }
                )

DS_CMSAF_ocean


<xarray.Dataset>
Dimensions:              (time: 45505, scene_across_track: 90, scene_channel: 4)
Coordinates:
  * scene_across_track   (scene_across_track) int32 1 5 9 13 ... 345 349 353 357
  * scene_channel        (scene_channel) int64 11 12 14 15
  * time                 (time) datetime64[ns] 2014-09-16 ... 2014-09-16T23:5...
    channel              (scene_channel) int32 12 13 15 16
Data variables:
    lat                  (time, scene_across_track) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    lon                  (time, scene_across_track) float64 dask.array<chunksize=(10, 10), meta=np.ndarray>
    eia                  (time, scene_across_track) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    sft                  (time, scene_across_track) float32 dask.array<chunksize=(10, 10), meta=np.ndarray>
    tb                   (time, scene_across_track, scene_channel) float32 dask.array<chunksize=(10, 10, 4), meta=np.ndarray>
    global_channel_ID    (scene_channel) int32 12 13 15 16
    central_freq         (scene_channel) float64 19.35 19.35 37.0 37.0
    polarization         (scene_channel) |S50 b'h' b'v' b'h' b'v'
    channel_uncertainty  (scene_channel) float32 2.4 1.27 3.0 1.34
Attributes:
    title:                 Environmental Scene 1
    comment:               feedhorn channels: h19, v19, v22
    elevation_offset_deg:  0.4
    azimuth_offset_deg:    -0.3

In [ ]:
delayed_obj = DS_CMSAF_ocean.to_netcdf(BT_dir2+BT_file2+"4chan"+".nc", compute=False)

from dask.diagnostics import ProgressBar
with ProgressBar():
     results = delayed_obj.compute()

In [ ]:
DS = xr.open_dataset(BT_dir+BT_file2+'.nc')
DS

In [ ]:
def rttovStrucInit():
    # Set knobs for use of OpenMP in RTTOV

    NthreadsF = 1  # Number of threads for forward model  #os.cpu_count()
    NthreadsFM = 4 # Number of threads for forward model (in case of multiple profiles, used in) #os.cpu_count()
    NprofsPerCallFM = 40 # ?Roughly speaking number of variables (e.g. 279) / Number of threads (e.g. 8)
    # Create instance of Profiles class; 
    # it's a container of the input atmospheric state that RTTOV will simulate

    nprofiles = 1  # This is hardcoded (we use RTTOV within OE, workin on a single profile per OE)
    nlev = 37 # TODO: this needs to be read from the apriori data (covariances and means) 
    myProfiles = pyrttov.Profiles(nprofiles, nlev)

    #  Create instance of RTTOV
    ssmiRttov = pyrttov.Rttov()

    #chan_list_ssmi = (12,13,14,15,16,17,18) #(1,12,13,14,15,16) #  #

    # Define instrument (FileCoef):
    ssmiRttov.FileCoef = '{}/{}'.format(rttov_installdir,
                                    "rtcoef_rttov13/rttov7pred54L/rtcoef_dmsp_16_ssmis.dat")

    # Load the instruments: for HIRS and MHS do not supply a channel list and
    # so read all channels
    try:
        ssmiRttov.loadInst() #chan_list_ssmi
    except pyrttov.RttovError as e:
        sys.stderr.write("Error loading instrument(s): {!s}".format(e))
        sys.exit(1)

    # Some settings
    ssmiRttov.Options.AddInterp = True
    # ssmiRttov.Options.InterpMode = 2
    ssmiRttov.Options.CO2Data = False
    ssmiRttov.Options.VerboseWrapper = False
    ssmiRttov.Options.DoCheckinput = False
    ssmiRttov.Options.UseQ2m = False
    ssmiRttov.Options.ApplyRegLimits = True
    ssmiRttov.Options.Verbose = False
    ssmiRttov.Options.FastemVersion = 6
    ssmiRttov.Options.Switchrad = True
    ssmiRttov.Options.Nthreads = NthreadsF

    # *********************************
    # *********************************
    return myProfiles, ssmiRttov

In [ ]:
# ONLY FOR MULTIPLE PROFILES CALL TESTING

NthreadsFM = 4 # Number of threads for forward model (in case of multiple profiles, used in) #os.cpu_count()
NprofsPerCallFM = 40 # ?Roughly speaking number of variables (e.g. 279) / Number of threads (e.g. 8)


ssmiRttovM = pyrttov.Rttov()

# SSMIS:

ssmiRttovM.FileCoef = '{}/{}'.format(rttov_installdir,
                                    "rtcoef_rttov13/rttov7pred54L/rtcoef_dmsp_16_ssmis.dat")

ssmiRttovM.Options.AddInterp = True
#ssmiRttovM.Options.InterpMode = 2
ssmiRttovM.Options.CO2Data = False
ssmiRttovM.Options.VerboseWrapper = False
ssmiRttovM.Options.DoCheckinput = False
ssmiRttovM.Options.UseQ2m = False
ssmiRttovM.Options.ApplyRegLimits = True
ssmiRttovM.Options.Verbose = False
ssmiRttovM.Options.FastemVersion = 6 
ssmiRttovM.Options.Nthreads = NthreadsFM
#ssmiRttovM.Options.NprofsPerCall = NprofsPerCallFM
ssmiRttovM.Options.Switchrad = True

# Load the instruments: for HIRS and MHS do not supply a channel list and
# so read all channels
try:
    ssmiRttovM.loadInst()
except pyrttov.RttovError as e:
    sys.stderr.write("Error loading instrument(s): {!s}".format(e))
    sys.exit(1)


In [ ]:
DS_CMSAF['wind'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Near surface wind speed (NSWP)',
                    'units'     : 'm/s'
                    }
                )
DS_CMSAF['wind_err'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'NSWP uncertainty',
                    'units'     : 'm/s'
                    }
                )

DS_CMSAF['chiSquareTest1'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Optimal solution agrees with observation in Y space',
                    'units'     : 'True/False'
                    }
                )
DS_CMSAF['chiSquareTest2'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Observation agrees with prior in Y space',
                    'units'     : 'True/False'
                    }
                )
DS_CMSAF['chiSquareTest3'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Optimal solution agrees with prior in Y space',
                    'units'     : 'True/False'
                    }
                )
DS_CMSAF['chiSquareTest4'] = xr.DataArray(
                data   = np.full((DS_CMSAF.time.shape[0],DS_CMSAF.scene_across_track.shape[0]),np.nan),   # enter data here
                dims   = ['time','scene_across_track'],
                coords = {'time': DS_CMSAF.time, 'scene_across_track':DS_CMSAF.scene_across_track},
                attrs  = {
                    #'_FillValue': -999.9,
                    'description': 'Optimal solution agrees with priot in X space',
                    'units'     : 'True/False'
                    }
                )

In [ ]:
DS_CMSAF = DS_CMSAF.chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t}) 
DS_CMSAF

In [ ]:
# Add init function:


def init_retrieval_data(time_i, lat, lon, 
                        channels_list, BT,
                       #dir_bands,lat_bands, 
                        zenithAngle): #, myProfiles,
                       #ssmiRttov, ssmiRttovM=None):   

    #adate = supporting_routines_m.datetime64_to_datetime(time_i)
    #season = supporting_routines_m.date2season(adate)
    
    #if((lat!=np.nan)|(lon!=np.nan)):
    
    myProfiles, ssmiRttov = rttovStrucInit()  
    ssmiRttovM=None

    #aprioLowCapDir = '/home/mario/Data/Covariance_means/CDS_api_data/ERA5_data/lower_cap/'
    #aprioLowMidCapDir = '/home/mario/Data/Covariance_means/CDS_api_data/ERA5_data/lowmid_cap/'
    #aprioUpMidCapDir = '/home/mario/Data/Covariance_means/CDS_api_data/ERA5_data/upmid_cap/'
    #aprioUpCapDir = '/home/mario/Data/Covariance_means/CDS_api_data/ERA5_data/upper_cap/'
    
    aprioLowCapDir = '/nobackup/users/echeverr/data/ECMWF_era5/hourly_ERA5/mean_cov/lower_cap/'
    aprioLowMidCapDir = '/nobackup/users/echeverr/data/ECMWF_era5/hourly_ERA5/mean_cov/lowmid_cap/'
    aprioUpMidCapDir = '/nobackup/users/echeverr/data/ECMWF_era5/hourly_ERA5/mean_cov/upmid_cap/'
    aprioUpCapDir = '/nobackup/users/echeverr/data/ECMWF_era5/hourly_ERA5/mean_cov/upper_cap/'

    # list of directories contaning covariances and means; each dir contains one geographical zone 
    # Geo. zones are divided in latitude strips: [-90,-40), [-40,0), [0,+40), [+40,+90]; lon. [-180,+180) for all
    dir_bands = [aprioLowCapDir,aprioLowMidCapDir,aprioUpMidCapDir,aprioUpCapDir] 
    lat_bands = [[-90.0,-40.0],[-40.0,0.0],[0.0,40.0],[40.0,90.0]]

    #print(nprofiles)
    #print(time_i)
    #print(lat)
    #print(channel_list)
    #print(BT)
    #print(dir_bands)
    #print(lat_bands)
    #print(zenithAngle)
    #print(myProfiles)
    #print(ssmiRttov)
    
    
    months = (time_i.astype('datetime64[M]').astype(int) % 12 + 1)%12 // 3 + 1
    season = month2season_vec(months)
   
    
    # Load Atlases: (if any)    
    #ssmiRttov.SurfEmisRefl = np.zeros((
    #    4, nprofiles, len(channels_list)), dtype=np.float64)
            
    y_obs = pd.Series(BT,
                index=channels_list
                     )            
            
    # TODO Sy needs to be loaded from somewhere according to the channels_list: ***********
            
    # Channels 12-16 (values from Deblonde-English 2003) (sigma or std)
    y_noise = pd.Series(
                    [
                        2.4, 1.27, 1.44, #3.0, 1.34
                    ],
                    index=channels_list
    )
                            
    # Variance values > std**2
    S_y = pd.DataFrame(
                np.diag(y_noise.values**2),
                index=channels_list,
                columns=channels_list,
    )
    # TODO Sy needs to be loaded from somewhere according to the channels_list  ***********
            
    datetime_obs = time_i #.values
    salinity = 35 # hardcoded, not good; TODO Mario
            
    band = get_band(lat,dir_bands,lat_bands) # gets the directory for the band where 'lat' is located
    xa,Sa,xb,Sb = get_mean_covs(band, season) # gets mean and covariances for the band and season
            
    # reverse order in x and S (profile variables are reordered from low pressure to high) 
    Sb = Sb.iloc[::-1,::-1]
    Sa = Sa.iloc[::-1,::-1]
    xb = xb.iloc[::-1].squeeze() # convert means from dataframe to series
    xa = xa.iloc[::-1].squeeze()
            
    x_vars = xa.index.values
    b_vars = xb.index.values
            
    # Get the pressure per level ; TODO: this needs to be independent of the variable (e.g. xb in this case)
    xb_index = [float(i.split('_')[0]) for i in xb.index if i.endswith('temp')]
    Pressure = np.array(xb_index).reshape(len(xb_index),1)
                        
    nlev = len([i for i in xb.index if i.endswith('temp')]) # number of levels in profile quantities


    # forward_b_init fills "myProfiles" with the "fixed" parameters for the RTTOV simulation.
    # The forward model F(x,b), RTTOV in our case, has two "parameters": x and b
    # x is the state vector that is being retrieved (as such it is allowed to change during the retrieval)
    # b contains all other parameters that are fixed during the retrieval (everything else that is not being retrieved)

    forward_b_init(Pressure, salinity,
                   lat, lon, datetime_obs, zenithAngle, myProfiles)
    
    # Define dictionary of parameters for the forward model:

    forwardKwArgs = {"myProfiles_a" : myProfiles, 
                    "ssmiRttov_a" : ssmiRttov,
                    "channels_list":channels_list.tolist()}    
    

    
    # Define dictionary of parameters for the forward model (Multiple profiles case):
    
    if ssmiRttovM != None:   # if an RTTOV instance for multiple profiles has been defined
        
        # Initialize multiple profiles for using a single call to RTTOV
        # This is to be passed to the Jacobian function inside pyOpEst:
        # The Jacobian is needed per parameter (len(xa.index)+len(xb.index))

        nProfilesM = len(xa.index)+len(xb.index) # total number of parameters (x and b)
            
        # Load Atlases for Multiple profiles: (if any)    
        #ssmiRttovM.SurfEmisRefl = np.zeros((
        #    4, nProfilesM, len(channels_list)), dtype=np.float64) # RTTOVv12 used (2,nprof,nchan)
                   
        myProfilesM = pyrttov.Profiles(nProfilesM, nlev) # 

        #press2 = np.ones((nlev, nProfilesM))*Pressure # Pressure:(nlev,1)
      
        # Initialize profile datastructure for use in Jacobian computation:        
        forward_b_init(np.ones((nlev, nProfilesM))*Pressure, salinity,
                       lat, lon, datetime_obs, zenithAngle, myProfilesM) 
    
        forwardKwArgsM = {"myProfiles_a" : myProfilesM, 
        "ssmiRttov_a" : ssmiRttovM,
        "channels_list":channels_list.tolist()}
    else:
        forwardKwArgsM = None    # if not, then use only the single profile RTTOV instance;
                                 # this results in a much slower Jacobian calculation.
        
        
    # Create OE object:
    
    oe_ref = pyOE.optimalEstimation( # oe_1 if windDisambiguation used
        x_vars, # state variable names
        xa,  # a priori
        Sa, # a priori uncertainty
        channels_list,  # measurement variable names
        y_obs, # observations
        S_y, # observation uncertainty
        forwardRT, # forward Operator
        userJacobian=rttovK, # RTTOV's K model operator
        forwardKwArgs=forwardKwArgs, # additonal function arguments
        #multipleForwardKwArgs=forwardKwArgsM, # additonal function arguments for jacobian 
        #x_truth=x_truth, # true profile
        b_vars=b_vars,   # Parameter vector variable names
        b_p=xb,        # Parameter vector 
        S_b=Sb,        # Parameters error covariance matrix 
        perturbation=0.01,
        verbose=False
    )
    return oe_ref


#init_retrieval_data_vec = np.vectorize(init_retrieval_data, excluded=['nprofiles',
#                                                                       'channels_list',
#                                                                        'dir_bands',
#                                                                        'lat_bands',
#                                                                        'zenithAngle'
#                                                                       ],
#                                       otypes = [pyOE.pyOEcore.optimalEstimation])


In [ ]:


#chunk_size_time = 10
#chunk_size_s_a_t = 10
#DS_CMSAF_ocean = DS_CMSAF_ocean.chunk({"time": chunk_size_time, 
#                                       "scene_across_track": chunk_size_s_a_t})
#DS_CMSAF_ocean.chunk({"time": 100, 
#                                       "scene_across_track": 10})

In [ ]:
aa = np.full((DS_CMSAF_ocean.time.shape[0],DS_CMSAF_ocean.scene_across_track.shape[0]),np.nan)
print(aa.shape)

In [ ]:
# Load TB dataset:

DS_CMSAF_ocean_all = xr.open_dataset(BT_dir+BT_file2)
# nearest to user input in dataset:
init_date = DS_CMSAF_ocean_all.time.sel(time=init_date, method = "nearest")
end_date = DS_CMSAF_ocean_all.time.sel(time=end_date, method = "nearest")

DS_CMSAF_ocean = DS_CMSAF_ocean_all.sel(time=slice(init_date,end_date)
                                       ).chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t})
DS_CMSAF_ocean

In [ ]:
def month2season(month):
    seasons = [
        'DJF',
        'MAM',
        'JJA',
        'SON',
    ]
    season = seasons[month-1] # month is 1 based index while seasons array needs 0 based indexing
   
    return season

month2season_vec = np.vectorize(month2season, otypes=[str])

In [ ]:
def testFunc(tb):
    tb2 = tb[0]
    return tb2

def retrieveWind(time, tb, global_channel, 
                 lat, lon, eia, 
                ):
    
   
    wind = np.full(lat.shape,np.nan)
    wind_err = np.full(lat.shape,np.nan)

    if(~np.isnan(lat)):  # Input dataset has been filtered for sea-land flag (so both, lat and lon are valid or nan)
        
        oe_ref = init_retrieval_data(time,
                                 lat, lon,
                                 global_channel, 
                                 tb, #dir_bands,lat_bands,
                                 eia) #, myProfiles,
                                 #ssmiRttov)
    
        #print(oe_ref)
        oe_ref.doRetrieval()     
    
        if oe_ref.converged:  # 
            _, _, w10m, _, _,_ = supporting_routines_m.splitX_all_2W(oe_ref.x_op)
            _, _, w10m_err, _, _,_ = supporting_routines_m.splitX_all_2W(oe_ref.x_op_err)
    
            wind =  w10m.values.item()
            wind_err =  w10m_err.values.item()
                
            #chiSquareTest1 = oe_ref.chiSquareTest()[0][0]
            #chiSquareTest2 = oe_ref.chiSquareTest()[0][1]
            #chiSquareTest3 = oe_ref.chiSquareTest()[0][2]
            #chiSquareTest4 = oe_ref.chiSquareTest()[0][3]
            
        #else:  # 
        #    wind = np.nan
        #    wind_err = np.nan        
            
            #_, _, u10m, v10m, _, tsk,_ = supporting_routines_m.splitX_all_2(oe_ref.x_op)
            #_, _, u10m_err, v10m_err, _, tsk_err,_ = supporting_routines_m.splitX_all_2(oe_ref.x_op_err)
            
            #w10m, w10m_err = supporting_routines_m.UV2Wvar(oe_ref.S_op.loc['00000_u10','00000_u10'],
            #                                      oe_ref.S_op.loc['00000_v10','00000_v10'],
            #                                      oe_ref.S_op.loc['00000_u10','00000_v10'],
            #                                      u10m.values.item(), v10m.values.item())
    
    # Not over ocean:
    #else:
        #print("Lat:")
        #print(lat)
        #print("Lon:")
        #print(lon)
        #wind = np.nan
        #wind_err = np.nan  
        
    return np.array([wind, wind_err,]) 

            

In [ ]:
#os.environ["MALLOC_TRIM_THRESHOLD_"] = "0" #"65536" #
#from dask.distributed import Client, progress
#client = Client()
#client
DS_CMSAF_ocean


In [ ]:
DS_CMSAF_ocean_grouped = DS_CMSAF_ocean.groupby("time.hour")
for hour_name, hour_group in DS_CMSAF_ocean_grouped:
    print(hour_group)

In [ ]:

os.environ["MALLOC_TRIM_THRESHOLD_"] = "0" #"65536" #
from dask.distributed import Client, progress, LocalCluster
    #client = Client(
    #    threads_per_worker=2, n_workers=8, processes=True)
    #client

startTimeAll = time.time()

DS_CMSAF_ocean_grouped = DS_CMSAF_ocean.groupby("time.hour")
i=0
for hour_name, hour_group in DS_CMSAF_ocean_grouped:
    
    startTimeHour = time.time()
    
    hour_group = hour_group.chunk({"time": chunk_size_time,
                                   "scene_across_track": chunk_size_s_a_t})
    #client = Client(
    #    threads_per_worker=2, n_workers=8, processes=True)
    #client

    with LocalCluster(threads_per_worker=2, 
                      n_workers=8, processes=True
                     ) as cluster, Client(cluster) as client:
        
        out_wind = xr.apply_ufunc(retrieveWind, 
                              hour_group.time, 
                              hour_group.tb, 
                              hour_group.global_channel_ID, 
                              hour_group.lat, hour_group.lon, 
                              hour_group.eia, 
                              input_core_dims=[[], 
                                               ["scene_channel"], 
                                               ["scene_channel"],[],[],[],#[],[],#[],[], 
                                               #[],[],[],[] #,[],[], 
                                              ], 
                              exclude_dims= set(("scene_channel",)),
                              output_core_dims=[
                                  ["results"], 
                              ], 
                              dask="parallelized", 
                              output_dtypes=[hour_group.wind.dtype], 
                              output_sizes={"results": 2},
                              vectorize=True 
                             ).compute().chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t})

    print("%.2f s , Time_Hour" % (time.time()-startTimeHour)) 

    if(i==0):
        out_wind_1 = out_wind    
    else:
        out_wind_1 = xr.concat((out_wind_1, out_wind), dim = "time")

    print("Time group:")
    print(i)
    print("Done!")
    i+=1
    
DS_CMSAF_ocean["wind"].data = out_wind_1.data[:,:,0]
DS_CMSAF_ocean["wind_err"].data = out_wind_1.data[:,:,1]

print("%.2f s , TimeAll_hour" % (time.time()-startTimeAll))

In [ ]:
startTimeAll = time.time()

out_wind = xr.apply_ufunc(retrieveWind, 
                              DS_CMSAF_ocean.time, 
                              DS_CMSAF_ocean.tb, 
                              DS_CMSAF_ocean.global_channel_ID, 
                              DS_CMSAF_ocean.lat, DS_CMSAF_ocean.lon, 
                              DS_CMSAF_ocean.eia, 
                              #DS_CMSAF_ocean.wind, DS_CMSAF_ocean.wind_err, 
                              #DS_CMSAF_ocean.chiSquareTest1,DS_CMSAF_ocean.chiSquareTest2, 
                              #DS_CMSAF_ocean.chiSquareTest3, DS_CMSAF_ocean.chiSquareTest4, 
                              input_core_dims=[[], 
                                               ["scene_channel"], 
                                               ["scene_channel"],[],[],[],#[],[],#[],[], 
                                               #[],[],[],[] #,[],[], 
                                              ], 
                              exclude_dims= set(("scene_channel",)),
                              output_core_dims=[#["time"], 
                                  ["results"], 
                                  #[], 
                                  #[], 
                                  #[], 
                                  #[]], 
                              ], 
                              dask="parallelized", 
                              output_dtypes=[DS_CMSAF_ocean.wind.dtype], 
                              output_sizes={"results": 2},
                              vectorize=True 
                             ).compute().chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t})

print("%.2f s , TimeAll" % (time.time()-startTimeAll)) 

DS_CMSAF_ocean["wind"].data = out_wind.data[:,:,0]
DS_CMSAF_ocean["wind_err"].data = out_wind.data[:,:,1]

In [ ]:
#from dask.diagnostics import ProgressBar

# or distributed.progress when using the distributed scheduler
delayed_obj = DS_wind.chunk(chunks={'time':10}).to_netcdf("DS_wind.nc", compute=False)

with ProgressBar():
     results = delayed_obj.compute()

In [ ]:
out_wind = out_wind.chunk({"time": chunk_size_time, 
                                       "scene_across_track": chunk_size_s_a_t})
#DS_CMSAF_ocean["wind"] 
DS_CMSAF_ocean["wind"].data = out_wind.data[:,:,0]
#DS_CMSAF_ocean["wind_err"] = out_wind.data[:,:,0]
DS_CMSAF_ocean["wind"]

In [ ]:
DS_CMSAF_ocean["wind"].data = out_wind_1.data[:,:,0]
DS_CMSAF_ocean["wind_err"].data = out_wind_1.data[:,:,1]

In [ ]:
out_wind_1[1000:2000,:,0]

In [ ]:
def mapPlotScat(x,y,data,namefile, mini, maxi, orthoCenter=None):
    # Make a Mercator map of the data using Cartopy
    
    fig = plt.figure()
    
    #ortho = ccrs.Orthographic(0,-15) # ccrs.Orthographic(60,-15)
    ortho = ccrs.PlateCarree()
    ax = plt.axes(projection=ortho)
    
    #crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
    geo = ccrs.PlateCarree() #ccrs.Geodetic()
    #crs = ccrs.Orthographic(60,-15)
    
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black',linewidth=0.1)
    
    xy = ortho.transform_points(geo, x, y)

    ax.set_global()
    ax.gridlines()    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.07, 
                      color='black', alpha=0.5, linestyle='--', draw_labels=True)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER  
    

    # Plot the air temperature as colored circles and the wind speed as vectors.
    im = ax.scatter(
        xy[:,0],
        xy[:,1],
        c=data,
        s=0.01, #0.15
        marker = "o",
        cmap="viridis",
        #transform=crs,
        vmin=mini, vmax=maxi,  # 3,18
        #vmin=130, vmax=270,  # 180, 270        
    )
    fig.colorbar(im).set_label("10m Wind Speed, RadEst [m/s]")
    #fig.colorbar(im).set_label("Temp. Bright [K]")
    
# Use an utility function to add tick labels and land and ocean features to the map.

    plt.tight_layout()
    #plt.show()
    plt.savefig(namefile+'.png', bbox_inches='tight', dpi=450)  

In [ ]:
area_def_world = load_area('areas.yaml', 'worldeqc30km70')# 'worldeqc30km70') # for plots
grid_lons_world, grid_lats_world = area_def_world.get_lonlats()

swath_radEst = SwathDefinition(lons=DS_CMSAF_ocean.lon.values, lats=DS_CMSAF_ocean.lat.values)
lons_radEst, lats_radEst = swath_radEst.get_lonlats()

world_lons, world_lats, world_wind_radEst = \
                           data_reduce.swath_from_lonlat_grid(grid_lons_world, grid_lats_world,
                            lons_radEst, lats_radEst, DS_CMSAF_ocean.wind.values,
                            radius_of_influence=1000)
world_lons, world_lats, world_wind_err_radEst = \
                           data_reduce.swath_from_lonlat_grid(grid_lons_world, grid_lats_world,
                            lons_radEst, lats_radEst, DS_CMSAF_ocean.wind_err.values,
                            radius_of_influence=1000)

In [ ]:
mapPlotScat(world_lons, world_lats, world_wind_radEst,
                 'World_wind_RadEst_Plat', 3,18)

In [ ]:
mapPlotScat(world_lons, world_lats, world_wind_err_radEst,
                 'World_wind_Err_RadEst_Plat',0.5,1.3)

In [ ]:
out_wind_1

In [ ]:
def get_TB_frame(BT_scene_env1, area_interest, begin_t, end_t):
    
    lat_scene1 = supporting_routines_m.generate_masked_array(BT_scene_env1.lat[begin_t:end_t,:],
                                                  BT_scene_env1.sft[begin_t:end_t,:], 0, '==', drop= False) 
    lon_scene1 = supporting_routines_m.generate_masked_array(BT_scene_env1.lon[begin_t:end_t,:],
                                                  BT_scene_env1.sft[begin_t:end_t,:], 0, '==', drop= False) 
    tb_scene1 = supporting_routines_m.generate_masked_array(BT_scene_env1.tb[begin_t:end_t,:,:],
                                                  BT_scene_env1.sft[begin_t:end_t,:], 0, '==', drop= False) 

    grid_lons_interest, grid_lats_interest = area_interest.get_lonlats()

    swath_scene1 = SwathDefinition(lons=lon_scene1, lats=lat_scene1)
    lons_scene1, lats_scene1 = swath_scene1.get_lonlats()

    reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1 = \
                           data_reduce.swath_from_lonlat_grid(grid_lons_interest, grid_lats_interest,
                            lons_scene1, lats_scene1, tb_scene1.values,
                            radius_of_influence=3000)

    return reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1
    #swath_reduced_scene1 = SwathDefinition(reduced_lons_scene1, reduced_lats_scene1)

In [ ]:
def defineArea(corners, proj_id, datum):
    #corners=parseMeta(data_name)

    lat_0 = '{lat_0:5.2f}'.format_map(corners)
    lon_0= '{lon_0:5.2f}'.format_map(corners)
    lon_bbox = [corners['min_lon'],corners['max_lon']]
    lat_bbox = [corners['min_lat'],corners['max_lat']]
#    area_dict = dict(datum=datum,lat_0=lat_0,lon_0=lon_0,
#                proj=proj_id,units='m')

    area_dict = dict(datum=datum,lat_0=-15,lon_0=60,
                proj=proj_id,units='m',a=6370997.0,)

    prj=pyproj.Proj(area_dict)
    x, y = prj(lon_bbox, lat_bbox)
    xsize=200
    ysize=200
    area_id = 'granule'
    area_name = 'modis swath 5min granule'
    area_extent = (x[0], y[0], x[1], y[1])
    print(area_extent)
    area_def = AreaDefinition(area_id, area_name, proj_id, 
                                   area_dict, xsize, ysize,area_extent)
    return area_def

In [ ]:
# Creation of area of interest:
#corners = {"min_lon": 25 , "max_lon": 75, "min_lat": -30 , "max_lat": 0, "lat_0": 60, "lon_0":-15}
corners = {"min_lon": -95 , "max_lon": 20, "min_lat": 3 , "max_lat": 50, "lat_0": 27, "lon_0":-57}
proj_id = 'eqc'  # eqc
datum = 'WGS84'
area_interest = defineArea(corners, proj_id, datum)

In [ ]:

area_def_world = load_area('areas.yaml', 'worldeqc30km70')# 'worldeqc30km70') # for plots
grid_lons_world, grid_lats_world = area_def_world.get_lonlats()

In [ ]:
def timeMapAnimScat(BT_scene, BT_attributes, channel, area, 
                      init_date, nFrames, delta_hours, namefile):
    # Make a Mercator map of the data using Cartopy
    
    import matplotlib.animation as animation

    crs = area.to_cartopy_crs()
    
    #fig2 = plt.subplots(1,2) 
    fig = plt.figure(frameon=False) #figsize=(8, 6))
    fig.add_axes([0,0,1,1])
    ax = plt.axes(projection=crs)  
    ax.set_global()
    gl = ax.gridlines()  
    ax.set_title("TB "+namefile)
    #ax.set_title("Wind Speed "+namefile)
    
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black') 
    
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER 


    #delta_hours = 12
    end_date = init_date + np.timedelta64(delta_hours, 'h') 
    time_slice = np.where((BT_attributes.time.values>=init_date)&(
        BT_attributes.time.values<end_date))
    begin_t = time_slice[0][0]  
    end_t = time_slice[0][-1]

    x, y, data = get_TB_frame(BT_scene, area, begin_t, end_t)
    
    ims = []
    im1 = plt.scatter(
            x,
            y,
            c=data[:,channel],
            s=0.15,
            cmap="viridis",
            #transform=ccrs.PlateCarree(),
            #vmin=3, vmax=12     # 180, 270
            vmin=130, vmax=270     # 180, 270
        ) 
    
    #fig.colorbar(im1).set_label("10m Wind Speed [m/s]")
    #fig.colorbar(im1).set_label("Temp. Bright. [K]") 
    
    for i in np.arange(nFrames):

        im1 = plt.scatter(
            x,
            y,
            c=data[:,channel],
            s=0.15,
            cmap="viridis",
            transform=ccrs.PlateCarree(),
            #vmin=3, vmax=12  # 180, 270
            vmin=130, vmax=270  # 180, 270
        ) 
        
        ims.append([im1])
        init_date = init_date + np.timedelta64(delta_hours, 'h') 
        end_date = end_date + np.timedelta64(delta_hours, 'h') 
        time_slice = np.where((BT_attributes.time.values>=init_date)&(
            BT_attributes.time.values<end_date))
        begin_t = time_slice[0][0]  
        end_t = time_slice[0][-1] 
        x, y, data = get_TB_frame(BT_scene, area, begin_t, end_t)

        
    im_ani = animation.ArtistAnimation(fig, ims, interval=500, repeat_delay=3000,
                                   blit=True)
    # To save this second animation with some metadata, use the following command:
    # im_ani.save('im.mp4', metadata={'artist':'Guido'})
    plt.tight_layout()
    im_ani.save(namefile+'.mp4',dpi=150)
    plt.show()

In [ ]:
nFrames = 12
delta_hours = 12
channel = 2
namefile = 'env1_22V_12h_F16'
timeMapAnimScat(BT_scene, BT_attributes, channel, area_interest, 
                      init_date, nFrames, delta_hours, namefile)

In [ ]:
# Resample swath scene 1 at a world wide scale:

result_scene1 = resample_nearest(swath_scene1, tb_scene1.values, area_def_world, 
                          radius_of_influence=30000, fill_value=np.nan)

#result_scene2 = resample_nearest(swath_scene2, tb_scene2.values, area_def_world, 
#                          radius_of_influence=30000, fill_value=np.nan)

In [ ]:
# Mask out swath part that intersects the area of interest


#reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2 = \
#                           data_reduce.swath_from_lonlat_grid(grid_lons_interest, grid_lats_interest,
#                            lons_scene2, lats_scene2, tb_scene2.values,
#                            radius_of_influence=3000)
#swath_reduced_scene2 = SwathDefinition(reduced_lons_scene2, reduced_lats_scene2)

In [ ]:
# Mask out swath part that intersects the area of interest
world_lons_scene1, world_lats_scene1, world_data_scene1 = \
                           data_reduce.swath_from_lonlat_grid(grid_lons_world, grid_lats_world,
                            lons_scene1, lats_scene1, tb_scene1.values,
                            radius_of_influence=3000)
swath_world_scene1 = SwathDefinition(reduced_lons_scene1, reduced_lats_scene1)

world_lons_scene2, world_lats_scene2, world_data_scene2 = \
                           data_reduce.swath_from_lonlat_grid(grid_lons_world, grid_lats_world,
                            lons_scene2, lats_scene2, tb_scene2.values,
                            radius_of_influence=3000)
swath_world_scene2 = SwathDefinition(reduced_lons_scene2, reduced_lats_scene2)

In [ ]:
# Resample swath data into a grid in the area of interest 
result_reduced_scene1 = resample_nearest(swath_reduced_scene1, reduced_data_scene1, area_interest, 
                                  radius_of_influence=30000, fill_value=None)

result_reduced_scene2 = resample_nearest(swath_reduced_scene2, reduced_data_scene2, area_interest, 
                                  radius_of_influence=30000, fill_value=None)

In [ ]:
mapArea('mapArea0', area_def_world)
mapArea('mapArea01', area_interest)

In [ ]:
chann = 0
# Plot resampled (grid version) scenes:
basicMapPlot(result_scene1[:,:,chann],'scene1'+str(chann), area_def_world)  # map of the whole world, grid
basicMapPlot(result_reduced_scene1[:,:,chann],
             'scene1_reduced'+str(chann), area_interest)  # map only the area of interest, grid

basicMapPlot(result_scene2[:,:,chann],'scene2'+str(chann), area_def_world)  # map of the whole world, grid
basicMapPlot(result_reduced_scene2[:,:,chann],
             'scene2_reduced'+str(chann), area_interest)  # map only the area of interest, grid

In [ ]:
# Plot in area of interest, Plate Carree projection

chann = 0
# Plot original swath pixels:
basicMapPlotScat(reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1[:,chann],
                 'scene1_scatt_PCarr_19H', area_interest, )
chann = 1
# Plot original swath pixels:
basicMapPlotScat(reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1[:,chann],
                 'scene1_scatt_PCarr_19V', area_interest, )
chann = 2
# Plot original swath pixels:
basicMapPlotScat(reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1[:,chann],
                 'scene1_scatt_PCarr_22V', area_interest, )
#chann = 1
#basicMapPlotScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
#                 'scene2_scatt_PCarr_91V', area_interest)

In [ ]:
image_path = Path('/nobackup/users/echeverr/fortran_tests/netcdf/following_Edouard/imgs/ch19H')
images = list(image_path.glob('*.png'))
image_list = []
for file_name in images:
    image_list.append(imageio.imread(file_name))

In [ ]:
imageio.mimwrite('animated_from_images.gif', image_list)

In [ ]:
# Plot in world area, Orthographic projection

chann = 2
# Plot original swath pixels:
basicMapPlotScat1(reduced_lons_scene1, reduced_lats_scene1, reduced_data_scene1[:,chann],
                 'world_scene1_scatt_Orth_22V', area_interest)
#chann = 3
#basicMapPlotScat1(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
#                 'world_scene1_scatt_Orth_91H', area_interest)

In [ ]:
# Plot original swath pixels:
chann = 2
basicMapPlotScat1(world_lons_scene1, world_lats_scene1, world_data_scene1[:,chann],
                 'scene1_scatt_Orth_world_22V', area_def_world)
chann = 3
basicMapPlotScat1(world_lons_scene2, world_lats_scene2, world_data_scene2[:,chann],
                 'scene1_scatt_Orth_world_91H', area_def_world)

In [ ]:
def basicMapPlot(result,namefile, area):
    crs = area.to_cartopy_crs()
    fig, ax = plt.subplots(figsize=(13, 3))
    ax= plt.axes(projection=crs)
    #ax.background_img(name='BM', resolution='high') 
    ax.coastlines();
    #ax.stock_img();
    ax.grid(True)
    #ax.set_xlabel('Longitude [deg]')
    #ax.set_ylabel('Latitude [deg]')

    #gl = ax.gridlines(crs=ccrs.Orthographic(), linewidth=0.1, 
    #                  color='black', alpha=0.5, linestyle='--', draw_labels=True)
    #gl.xlabels_top = False
    #gl.ylabels_left = False
    #gl.ylabels_right=True
    #gl.xlines = True
    #gl.xlocator = mticker.FixedLocator([70, 75, 80, 85])
    #gl.ylocator = mticker.FixedLocator([-5, -3, -1, 1, 3])
    #gl.xformatter = LONGITUDE_FORMATTER
    #gl.yformatter = LATITUDE_FORMATTER

    im = ax.imshow(result, transform=crs, extent=crs.bounds, origin='upper', cmap='jet', vmin=150, vmax=250)
    fig.colorbar(im,ax=ax) 
    plt.savefig(namefile+'.png', bbox_inches='tight', dpi=150)  


In [ ]:
def basicMapPlotScat(x,y,data,namefile, area):
    # Make a Mercator map of the data using Cartopy
    
    crs = area.to_cartopy_crs()
    
    fig = plt.figure(figsize=(8, 6))
    #plt.figure(figsize=(8, 6))
    ax = plt.axes(projection=crs)   
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
    ax.set_global()
    ax.gridlines()        
    ax.set_title("TB")
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.1, 
                      color='black', alpha=0.5, linestyle='--', draw_labels=True)
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER    

    # Plot the air temperature as colored circles and the wind speed as vectors.
    im = ax.scatter(
        x,
        y,
        c=data,
        s=0.15,
        cmap="viridis",
        transform=ccrs.PlateCarree(),
        #vmin=3, vmax=18         #180, 270
        vmin=130, vmax=270         #180, 270
    )
    fig.colorbar(im).set_label("Brightness temperature [K]")
    
# Use an utility function to add tick labels and land and ocean features to the map.

    plt.tight_layout()
    #plt.show()
    plt.savefig(namefile+'.png', bbox_inches='tight', dpi=150)  

In [ ]:
def mapArea(namefile, area):

    crs = area.to_cartopy_crs()
    fig, ax = plt.subplots(figsize=(13, 3))
    ax= plt.axes(projection=crs)
    
    #gl = ax.gridlines(crs=ccrs.Orthographic(), linewidth=0.1, 
    #                  color='black', alpha=0.5, linestyle='--', draw_labels=True)

    #gl.xformatter = LONGITUDE_FORMATTER
    #gl.yformatter = LATITUDE_FORMATTER
    
    ax.coastlines(linewidth=0.5)   
    #ax.set_global() 
    plt.savefig(namefile+'.png', bbox_inches='tight', dpi=300)  

In [ ]:
#chann = 0
#basicMapAnimScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
#                 'scene1_scatt'+str(chann), area_interest)

nFrames = 150
start_frame = 0
nAcrossSwathFrame = 180

chann = 0
basicMapAnimScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_red_19H', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 2
basicMapAnimScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_red_91V', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 2
basicMapAnimScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_red_22V', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 3
basicMapAnimScat(reduced_lons_scene2, reduced_lats_scene2, reduced_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_red_91H', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

In [ ]:
def basicMapAnimScat(x,y,data,namefile, area, 
                      nFrames, start_frame, nAcrossSwathFrame):
    # Make a Mercator map of the data using Cartopy
    
    import matplotlib.animation as animation

    crs = area.to_cartopy_crs()
    
    #fig2 = plt.subplots(1,2) 
    fig = plt.figure(figsize=(8, 6))
    ax = plt.axes(projection=crs)  
    ax.set_global()
    gl = ax.gridlines()  
    #ax.set_title("TB "+namefile)
    ax.set_title("Wind Speed "+namefile)
    
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
   
    #gl0 = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=0.1, 
    #                  color='black', alpha=0.5, linestyle='--', draw_labels=True)

    #gl0.xformatter = LONGITUDE_FORMATTER
    #gl0.yformatter = LATITUDE_FORMATTER 
    
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER 
    
    start_ = start_frame
    end_ = nAcrossSwathFrame
    
    ims = []
    x2 = x[start_:end_]
    y2 = y[start_:end_]
    data2_0 = data[start_:end_]
    #data2_2 = data[start_:end_,2]

    im1 = plt.scatter(
            x2,
            y2,
            c=data2_0,
            s=0.15,
            cmap="viridis",
            #transform=ccrs.PlateCarree(),
            vmin=3, vmax=12     # 180, 270
            #vmin=130, vmax=270     # 180, 270
        ) 
    fig.colorbar(im1).set_label("10m Wind Speed [m/s]")
    #fig.colorbar(im1).set_label("Temp. Bright. [K]") 
    for i in np.arange(nFrames):
        #ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

        im1 = plt.scatter(
            x2,
            y2,
            c=data2_0,
            s=0.15,
            cmap="viridis",
            transform=ccrs.PlateCarree(),
            vmin=3, vmax=12  # 180, 270
            #vmin=130, vmax=270  # 180, 270
        ) 

        ims.append([im1])
      
        
        start_ +=nAcrossSwathFrame
        end_ +=nAcrossSwathFrame
        x2 = np.append(x2,x[start_:end_])
        y2 = np.append(y2,y[start_:end_])
        data2_0 = np.append(data2_0,data[start_:end_])
        #data2_2 = np.append(data2_2,data[start_:end_,2])

        
    im_ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=3000,
                                   blit=True)
    # To save this second animation with some metadata, use the following command:
    # im_ani.save('im.mp4', metadata={'artist':'Guido'})
    im_ani.save(namefile+'_bar.mp4',dpi=150)
    plt.show()

In [ ]:
BT_scene_env2 

In [ ]:
def basicMapPlotScat1(x,y,data,namefile, area):
    # Make a Mercator map of the data using Cartopy
    
    fig = plt.figure()
    
    ortho = ccrs.Orthographic(60,-15)
    ax = plt.axes(projection=ortho)
    
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
    geo = ccrs.Geodetic()
    #crs = ccrs.Orthographic(60,-15)
    
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
    
    xy = ortho.transform_points(geo, x, y)

    ax.set_global()
    ax.gridlines()    
    
    #ax.set_title("TB")
    #ax.coastlines() 
    # Plot the air temperature as colored circles and the wind speed as vectors.
    im = ax.scatter(
        xy[:,0],
        xy[:,1],
        c=data,
        s=0.15,
        cmap="viridis",
        #transform=crs,
        #vmin=3, vmax=18,  # 180, 270
        vmin=130, vmax=270,  # 180, 270        
    )
    #fig.colorbar(im).set_label("10m Wind Speed, HOAPS [m/s]")
    fig.colorbar(im).set_label("Temp. Bright [K]")
    
# Use an utility function to add tick labels and land and ocean features to the map.

    plt.tight_layout()
    #plt.show()
    plt.savefig(namefile+'.png', bbox_inches='tight', dpi=300)  

In [ ]:
nFrames = 1200
start_frame = 0
nAcrossSwathFrame = 180

chann = 0
basicMapAnimScat1(world_lons_scene1, world_lats_scene1, world_data_scene1[:,chann],
                 'An_scene1_scatt_Orth_world_19H', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 2
basicMapAnimScat1(world_lons_scene2, world_lats_scene2, world_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_world_91V', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 2
basicMapAnimScat1(world_lons_scene1, world_lats_scene1, world_data_scene1[:,chann],
                 'An_scene1_scatt_Orth_world_22V', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

chann = 3
basicMapAnimScat1(world_lons_scene2, world_lats_scene2, world_data_scene2[:,chann],
                 'An_scene1_scatt_Orth_world_91H', area_interest, 
                  nFrames, start_frame, nAcrossSwathFrame)

In [ ]:
def basicMapAnimScat1(x,y,data,namefile, area, 
                      nFrames, start_frame, nAcrossSwathFrame):
    # Make a Mercator map of the data using Cartopy
    
    import matplotlib.animation as animation

    fig = plt.figure(figsize=(8, 6))
    
    ortho = ccrs.Orthographic(-39,18) #ccrs.Orthographic(60,-15)
    ax = plt.axes(projection=ortho)
    
    crs = ccrs.RotatedPole(pole_longitude=177.5, pole_latitude=37.5)
    geo = ccrs.Geodetic()
    
    ax.add_feature(cartopy.feature.LAND, zorder=0, edgecolor='black')
    
    xy = ortho.transform_points(geo, x, y)

    ax.set_global()
    ax.gridlines()        
       
    ax.set_title("Wind Speed "+namefile) 
    #ax.set_title("Temperature Brightness "+namefile) 
    
    start_ = start_frame
    end_ = nAcrossSwathFrame
    
    ims = []
    x2 = xy[start_:end_,0]
    y2 = xy[start_:end_,1]
    data2_0 = data[start_:end_]
    #data2_2 = data[start_:end_,2]

    im1 = plt.scatter(
            x2,
            y2,
            c=data2_0,
            s=0.15,
            cmap="viridis",
            #transform=ccrs.PlateCarree(),
            vmin=3, vmax=18  # 180, 250
            #vmin=130, vmax=270   # 180, 270 TB
        ) 
    fig.colorbar(im1).set_label("10m Wind Speed [m/s]")
    #fig.colorbar(im1).set_label("Temp. Bright. [K]") 
    for i in np.arange(nFrames):
        #ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

        #plt.tight_layout()
        im1 = plt.scatter(
            x2,
            y2,
            c=data2_0,
            s=0.15,
            cmap="viridis",
            #transform=ccrs.PlateCarree(),
            vmin=3, vmax=18   # 180, 270 TB
            #vmin=130, vmax=270   # 180, 270 TB
        ) 

        ims.append([im1])
      
        
        start_ +=nAcrossSwathFrame
        end_ +=nAcrossSwathFrame
        x2 = np.append(x2,xy[start_:end_,0])
        y2 = np.append(y2,xy[start_:end_,1])
        data2_0 = np.append(data2_0,data[start_:end_])
        #data2_2 = np.append(data2_2,data[start_:end_,2])

        
    im_ani = animation.ArtistAnimation(fig, ims, interval=30, repeat_delay=3000,
                                   blit=True)
    # To save this second animation with some metadata, use the following command:
    # im_ani.save('im.mp4', metadata={'artist':'Guido'})
    im_ani.save(namefile+'.mp4',dpi=150)
    plt.show()